In [19]:
pip install bs4

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/packaging/version.py", line 49, in parse
    return Version(version)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/packaging/version.py", line 266, in __init__
    raise InvalidVersion(f"Invalid version: '{version}'")
pip._vendor.packaging.version.InvalidVersion: Invalid version: '.dist-info'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/commands/install.py", line 345, in run
    session = self.get_default_session(options)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/

In [20]:
pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup
import tqdm

url = "https://mcpedl.org/post-sitemap.xml"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'xml')

versions = []
for url in soup.find_all('url'):
    loc = url.find('loc').text
    if loc.startswith("https://mcpedl.org/minecraft-pe-"):
        version = loc.split("https://mcpedl.org/minecraft-pe-")[1].split("/")[0]
        if "-apk" in version:
            version = version.split("-apk")[0]
        versions.append(version)

sorted_versions = sorted(versions, key=lambda x: [int(y) for y in x.split('-') if y.isdigit()])

downloadable_versions = []

def check_version(url):
    version = url.find('loc').text.split("https://mcpedl.org/minecraft-pe-")[1].split("/")[0]
    if "-apk" in version:
        version = version.split("-apk")[0]
    version_elegida = version.replace('-', '.')
    lastmod = url.find('lastmod').text.strip()[:10]
    fecha = '-'.join(reversed(lastmod.split('-')))
    url_final = f"https://mcpedl.org/uploads_files/{fecha}/minecraft-{version}.apk"
    response = requests.get(url_final, stream=True)
    if response.status_code == 200:
        content_length = response.headers.get("Content-Length")
        if content_length and int(content_length) > 1000:
            response.close()
            soup = BeautifulSoup(response.content, 'html.parser')
            if soup.title == "404 Not Found" in soup.title.string:
                response.close()
                return None
            else:
                version_parts = version_elegida.split('.')
                # pad version parts with leading zeros to ensure numerical sorting
                version_parts = [part.zfill(3) for part in version_parts]
                # join version parts back into a single string
                version_sort_key = ''.join(version_parts)
                return version_sort_key, version_elegida
        else:
            response.close()
            return None
    else:
        response.close()
        return None

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(check_version, url) for url in soup.find_all('url') if url.find('loc').text.startswith("https://mcpedl.org/minecraft-pe-")]
    for future in tqdm.tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        result = future.result()
        if result:
            version_sort_key, version_elegida = result
            downloadable_versions.append(version_elegida)

    # sort the versions numerically
    downloadable_versions = sorted(downloadable_versions, key=lambda v: [int(part) for part in v.split('.')])
    
    print(downloadable_versions)


version_elegida = input("\nEscribe la versión que deseas descargar (en el formato 1.2.3): ")
if version_elegida == "":
    # select the latest version available
    downloadable_versions.sort(reverse=True)
    version_elegida = downloadable_versions[0].replace('-', '.')
else:
    # check if the version entered by the user is available
    if version_elegida not in downloadable_versions:
        print(f"La versión {version_elegida} no está disponible para descargar.")
        exit()
    version_elegida = version_elegida.replace('.', '-')

# find the URL for the selected version
for url in soup.find_all('url'):
    loc = url.find('loc').text
    if loc.startswith(f"https://mcpedl.org/minecraft-pe-{version_elegida}"):
        lastmod = url.find('lastmod').text.strip()[:10]
        fecha = '-'.join(reversed(lastmod.split('-')))
        url_final = f"https://mcpedl.org/uploads_files/{fecha}/minecraft-{version_elegida}.apk"
        print(f"\nLa URL final es: {url_final}")
        break

100%|██████████| 399/399 [00:09<00:00, 42.08it/s]


['1.4.0', '1.4.1', '1.4.2', '1.16.0.67', '1.16.20.50', '1.16.20.53', '1.16.100.54', '1.16.100.55', '1.16.100.56', '1.16.100.57', '1.16.201', '1.16.210.51', '1.16.210.54', '1.16.210.55', '1.16.210.56', '1.16.210.57', '1.16.210.60', '1.16.210.61', '1.16.220.50', '1.16.220.51', '1.16.230.52', '1.16.230.54', '1.16.230.56', '1.17.0.50', '1.17.10', '1.17.11', '1.17.30', '1.17.30.21', '1.17.30.22', '1.17.30.23', '1.17.30.24', '1.17.30.25', '1.17.32', '1.17.34', '1.17.40.21', '1.17.40.23', '1.17.41', '1.18.0', '1.18.0.21', '1.18.0.24', '1.18.0.25', '1.18.0.27', '1.18.2', '1.18.10', '1.18.10.20', '1.18.10.21', '1.18.10.22', '1.18.10.27', '1.18.12', '1.18.20.27', '1.18.30', '1.18.30.32', '1.18.31', '1.18.32', '1.19.0', '1.19.0.20', '1.19.0.24', '1.19.0.26', '1.19.0.28', '1.19.0.30', '1.19.0.32', '1.19.0.34', '1.19.2', '1.19.10', '1.19.10.22', '1.19.10.23', '1.19.10.24', '1.19.20', '1.19.20.20', '1.19.20.23', '1.19.21', '1.19.22', '1.19.30.21', '1.19.30.22', '1.19.30.23', '1.19.30.25', '1.19.31',